# Implementing contact surveys
Next, let's think about how we might actually use empiric data
to inform a heterogeneous mixing model.
This is something that we need to be careful about,
because it would be easy to make mistakes or use
assumptions that don't make sense.
Of course, the first thing to understand is what the empiric data
that we're working with represent.

## How contact surveys gather data
Typically when conducting a contact survey,
participants are asked to record or remember how many
contacts they had with people of certain characteristics 
over the course of a period of time.
For example, the "POLYMOD" survey 
(published by Mossong et al. in the journal _PLOS Medicine_)
was a seminal study in the field
which asked people from eight European countries to record
the number of contacts they had over a 24-hour period.
Participants recorded the number of contacts they had over this period,
as well as characteristics of the persons they contacted
and the setting in which the contact occurred.
To keep things simpler, let's just think about the number
of contacts each survey participant had according to the 
age of the person that they came into contact with.

## The nature of survey data
The first, and most important thing to note about these data
is that the number of contacts recorded is fundamentally different
from the numbers we were using in the previous notebook,
which assumed density-dependent transmission.
Specifically, because the surveys ask how many people
of a certain characteristic the respondent is coming into contact with,
the matrices are not _per capita_ from the perspective of the contacted person,
only from the perspective of the respondent/contactor.

To reason about this, let's start off just dividing our population
into two groups: adults and children.
This is not the starting format of the POLYMOD data,
but this assumption should help keep things simple for now.

For example, suppose a child responds to a survey
to say that they contacted three other children during the survey period,
this gives us some evidence that children in general may contact
approximately three other children in a 24-hour period,
but we would need to do further calculation to estimate the 
average rate at which two specific children would come into contact.

By contrast to the previous notebook,
we are now essentially dealing with frequency-dependent contact rates.
Other than this consideration,
we'll keep all our assumptions consistent with the ones
we were using in the previous notebook, including:
- The matrices contain rates of social contact 
(so we'll use the infection flow's parameter to define risk of transmission per contact)
- Population sub-groups are mutually exclusive and cover the full modelled population
- Susceptibility and infectiousness will be addressed outside of the matrix (or not varied between groups)
- Sub-groups will be ordered sequentially left to right and top to bottom

## Why the values are different
We'll work through this question in more detail.
However, to gain some immediate intuition,
note that with the current assumptions the rate at which a person from one sub-group (e.g. children)
contacts other persons from some other sub-group of the population (e.g. other children)
is dependent (at least to some extent) to the number of people from the other sub-group in the population.
By constrast, the rate at which two specific people come into contact is not.

To take this to an extreme, consider an island on which the passengers of ship have been marooned,
with the passengers consisting of 99 adults and one child.
If we surveyed the child to ask how many other children she contacted in a recent 24-hour period, 
her answer would clearly be zero.
So we can see that the numbers we get back from these surveys are likely
to be influenced by the size of the populations available for contact.

## Comparing to the mixing and transmission notebook
In the previous notebook `mixing-and-transmission-types`,
we had a population size of one and a matrix that contained values less than one.
This was done just in order to demonstrate that it was possible 
to achieve equivalence between the density- and frequency-dependent assumptions 
under certain parameter configurations.
However, the values we used were not intended to be realistic.

Let's get started with a frequency-dependent transmission model 
that is still a pretty coarse simplification of any real population,
but uses numbers that are slightly closer to reality.
As a starting point, the POLYMOD study found that the average number of contacts
reported by each survey respondent per 24-hour period was about 13,
but let's use 12 as a slightly rounder number.
Let's also start off ignoring the results of the POLYMOD study 
or any other empiric evidence for contact structures.
If we consider that one third of the total population are children,
we might build the following mixing matrix:

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.plotting.backend = "plotly"
from jax import numpy as jnp

from summer2 import CompartmentalModel, Stratification
from summer2.parameters import Parameter, Function

In [ ]:
def build_sir_model(
    config: dict,
) -> CompartmentalModel:
    """
    Similar model construction function as in mixing-and-transmission-types notebook,
    except adding in frequency-dependent transmission.
    """
    compartments = config["compartments"]
    analysis_times = (0.0, config["end_time"])
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=("infectious",),
    )
    model.set_initial_population(
        distribution=
        {
            "susceptible": config["population"] - config["seed"], 
            "infectious": config["seed"],
        }
    )
    
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("risk_per_contact"),
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=1. / Parameter("infectious_period"),
        source="infectious", 
        dest="recovered",
    )
    
    model.request_output_for_compartments(
        "prevalence",
        "infectious",
    )
    
    return model

In [ ]:
def build_simple_strat(
    compartments: list,
    mixing_matrix: jnp.array,
) -> Stratification:
    """
    Same stratification construction function as in mixing-and-transmission-types notebook.
    """
                
    mix_strat = Stratification(
        "age",
        ["child", "adult"],
        compartments,
    )
    
    prop_child = Parameter("prop_child")
    prop_adult = 1. - prop_child
    mix_strat.set_population_split(
        {
            "child": prop_child,
            "adult": prop_adult,
        }
    )

    mix_strat.set_mixing_matrix(mixing_matrix)

    return mix_strat

In [ ]:
model_config = {
    "end_time": 40.0,
    "population": 1.0,
    "seed": 0.01,
    "compartments": ("susceptible", "infectious", "recovered"),
}

In [ ]:
parameters = {
    "risk_per_contact": 0.05,
    "infectious_period": 4.0,
    "prop_child": 1.0 / 3.0,
    "total_contacts": 13.0,
}

In [ ]:
def build_frequency_mixing_matrix(prop_group1, total_contacts):
    return total_contacts * jnp.array(
        [
            [prop_group1, 1.0 - prop_group1],
            [prop_group1, 1.0 - prop_group1],
        ]
    )

## Demonstrating equivalence


In [ ]:
outputs = pd.DataFrame()

strat_model = build_sir_model(model_config)
mixing_matrix = Function(
    build_frequency_mixing_matrix, 
    (Parameter("prop_child"), Parameter("total_contacts")),
)
mix_strat = build_simple_strat(model_config["compartments"], mixing_matrix)
strat_model.stratify_with(mix_strat)

strat_model.run(parameters)
outputs["stratified"] = strat_model.get_derived_outputs_df()["prevalence"]

In [ ]:
unstrat_params = {
    "infectious_period": parameters["infectious_period"],
    "risk_per_contact": parameters["risk_per_contact"] * parameters["total_contacts"],
}
unstrat_model = build_sir_model(model_config)
unstrat_model.run(unstrat_params)
outputs["unstratified"] = unstrat_model.get_derived_outputs_df()["prevalence"]

In [ ]:
outputs.plot()